# Multirotor Dynamic Model

In [2]:
import casadi as ca
import numpy as np
import sys
sys.path.insert(0, '../python/pyecca')
import pyecca.lie.so3 as so3

Here we will develop a simple thrust model for the propeller.

In [3]:
def thrust(throttle, rho, r, V, kV, CT):
    """
    throttle: 0-1 controls voltage, []
    rho: density of air, [kg/m^3
    r: radius of propeller, [m]
    V: voltage of battery [V]
    kV: motor kV constant, [rpm/V]
    """
    omega = throttle*V*kV*(2*ca.pi/60)
    q = 0.5*rho*(omega*r)**2
    s = ca.pi*r**2
    return CT*q*s

thrust(throttle=1, rho=1.225, r=0.05, V=11.1, kV=1550, CT=1.0e-2)

0.39039349535108125

In [31]:
# state space model
#x: state: (position, orientation, velocity (body frame), angular velocity (body frame))
#            3          mrp,4           3                      3
x = ca.SX.sym('x',13)
pos_n = x[0:3]
r_nb = x[3:7]
vel_b = x[7:10]
omega_b = x[10:13]

#u: control
u = ca.SX.sym('u', 4)

#y: output, just state for now
y = x

#parameters: Jx Jy Jz Jxy g m CL0 CLa CD0
p = ca.SX.sym('J',9)
Jx = p[0]
Jy = p[1]
Jz = p[2]
Jxz = p[3]
g = p[4]
m = p[5]
CL0 = p[6]
CLa = p[7]
CD0 = p[8]

#inertia tensor
J_b = ca.SX.zeros(3,3)
J_b[0,0] = Jx
J_b[1,1] = Jy
J_b[2,2] = Jz
J_b[0,2] = J_b[2,0] = Jxz

C_nb = so3.Dcm.from_mrp(r_nb) # DCM rotation
g_n = ca.vertcat(0,0,g) # gravity
vel_n = ca.mtimes(C_nb, vel_b) # inertial velocity


l_arm = 0.5
arm_angles = [45, -135, -34, 135]
for arm_angle in arm_angles:
    print(arm_angle)

thrusts = []
F_b = ca.vertcat(0,0,0)
M_b = ca.vertcat(0,0,0)

for i in range(u.size()[0]):
    thrust_i = thrust(throttle=u[i], rho=1.225, r=0.05, V=11.1, kV=1550, CT=1.0e-2)
    thrusts.append(thrust_i)
    angle = arm_angles[i]
    arm_vect = np.array([np.cos(angle),-np.sin(angle),0])
    thrust_vect = ca.vertcat(0,0,-thrust_i)
    M_b += ca.cross(arm_vect,thrust_vect)
    F_b += thrust_vect
m = 0.2;    
F_b += ca.mtimes(C_nb.T, m*g_n)
M_b

force_moment = ca.Function('force_moment',[x,u,p],[F_b,M_b],['x','u','p'],['F_b','M_b'])

#left hand side, xdot
xdot = ca.Function('rhs',[x,u,p],[ca.vertcat(
    ca.mtimes(C_nb, vel_b),
    so3.Mrp.kinematics(r_nb,omega_b),
    F_b/m - ca.cross(omega_b,vel_b),
    ca.mtimes(ca.inv(J_b), M_b - ca.cross(omega_b, ca.mtimes(J_b, omega_b))))],
    ['x','u','p'],['xdot'])

45
-135
-34
135


# Kinematics

Consider the dynamics

v = $\begin{bmatrix}
a
\end{bmatrix}$

In [16]:
def SO3_adjoint(V):
    v = V[0:3]
    w = V[3:6]